# Installing open-clip library

In [ ]:
%%capture
!pip install open_clip_torch

# Cloning GitHub Repository

In [ ]:
%%capture
!git clone https://github.com/kk-digital/kcg-ml.git

# OpenClip Examples

## Imports

In [ ]:
import torch
from PIL import Image
import open_clip

## Listing All Pre-Trained Models.

In [ ]:
open_clip.list_pretrained()

## Text-Image Matching Example.
Showing the probaility distribution of a list of texts for single image.

In [ ]:
MODEL_NAME = 'ViT-L-14'
PRETRAINED = 'openai'

In [ ]:
model, _, preprocess = open_clip.create_model_and_transforms(model_name=MODEL_NAME, pretrained=PRETRAINED)
tokenizer = open_clip.get_tokenizer(MODEL_NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
image = preprocess(Image.open('./kcg-ml/datasets/test-images/example3.jpg')).unsqueeze(0)

text = tokenizer(["pixel art", "painting", "digital art"]) # List of texts which will be compared.

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)

## Getting CLIP Image Embeddings for Single Image

In [ ]:
with torch.no_grad():
    image = preprocess(Image.open('./kcg-ml/datasets/test-images/example3.jpg')).unsqueeze(0).to(device)
    emb = model.encode_image(image).cpu().detach().numpy()

print(f"[INFO] CLIP embedding size: {emb.shape}")

## Checking the Similarity Between Two Image Using CLIP

In [ ]:
image1 = preprocess(Image.open('./kcg-ml/datasets/test-images/example1.jpg')).unsqueeze(0).to(device)
image2 = preprocess(Image.open('./kcg-ml/datasets/test-images/example2.jpg')).unsqueeze(0).to(device)

image_features = model.encode_image(image1)
image_2_features = model.encode_image(image2)

image_features /= image_features.norm(dim=-1, keepdim=True)
image_2_features /= image_2_features.norm(dim=-1, keepdim=True)
similarity = image_2_features.detach() @ image_features.detach().T
print(f'Similarit: {similarity.cpu().detach().numpy()[0][0]:.4f}')

## ClipModel Examples
▶ ClipModel : Module built over OpenClip function, check: https://github.com/kk-digital/kcg-ml/blob/main/examples/ClipTools.py

In [ ]:
!pip install patool

In [ ]:
import sys
sys.path.insert(0, './kcg-ml/clip_linear_probe_pipeline/')
sys.path.insert(0, './kcg-ml/')

from examples.ClipTools import ClipModel

## Creating an Instance of ClipModel Class

In [ ]:
clip_model_instance = ClipModel(clip_model=MODEL_NAME, pretrained=PRETRAINED)

## Downloading Model Method Example.

In [ ]:
clip_model_instance.download_model(MODEL_NAME, PRETRAINED)

## Getting Image Embedding Example.

In [ ]:
clip_model_instance.encode_image_from_image_file('./kcg-ml/datasets/test-images/example1.jpg')

In [ ]:
def image_to_bytes(image_path):
    with open(image_path, 'rb') as image_file:
        bytes_array = bytearray(image_file.read())
    return bytes_array

clip_model_instance.encode_image_from_image_data(image_to_bytes('./kcg-ml/datasets/test-images/example1.jpg'))

In [ ]:
clip_model_instance.encode_image_list(['./kcg-ml/datasets/test-images/example1.jpg', './kcg-ml/datasets/test-images/example2.jpg', './kcg-ml/datasets/test-images/example3.jpg'])